In [1]:
# !pip install transformers

In [1]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.optim import Adam, lr_scheduler
import torch.nn.functional as F

In [2]:
# !nvidia-smi

In [3]:
# from google.colab import drive
# drive.mount('./gdrive')

In [4]:
# cd /content/gdrive/My\ Drive/DeepLearning

In [5]:
# train_df = pd.read_csv('./files/train1.csv')
# test_df = pd.read_csv('./files/test1.csv')
# train_df.head(20)

In [9]:
# train_df = train_df.groupby('label_id').filter(lambda g: len(g) >= 100 and len(g) < 1000)
# train_df.groupby('label_id').count()

In [10]:
# train_df = train_df.groupby('병원').get_group('D')
# train_df.head(10)

In [2]:
class PetDataset(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    text = self.df.iloc[idx, 2]
    label = self.df.iloc[idx, 4]
    return text, label

In [3]:
train_df = pd.read_csv('./files/pre-fine.csv')
# test_df = pd.read_csv('./files/test1.csv')
train_df.head()

,질병명,주진단코드,SE,진단코드,label_id
0,당뇨병,2244,s 다음다뇨 식욕부진 최근들어 다음다뇨 증상과 month 전부터는 식욕도 약간 떨어...,당뇨병,15
1,간 효소 상승 (원인 미정),2081,s 건강검진 진행함 혈액검사 간수치 상승 alp alt 모두 상승함 영상검사 신장의...,간질환,1
2,당뇨병,2244,money 선결제 하셨습니다. 송이 식욕부진 구토 month 전부터 식욕부진 구토...,당뇨병,15
3,빈혈 (원인 미정),2231,s 체중감소 건강검진 혈액검사 빈혈 진행중 염증수치 상승 복부초음파검사 담낭내 점액...,빈혈,19
4,부 신피질 기능 항진증 · 커싱 증후군,2245,털빠짐 켁켁거림 나이가 좀 있어서 추가 검사 원하심 켁켁거리는 건 꽤 오래되었음 y...,신피질 커싱,24


In [11]:
train_dataset = PetDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda")
# device = torch.device('cpu')
# pretrained_path = './pretrained_without_wiki'
pretrained_path = './pretrained'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_path, do_lower_case=False)
model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=39)
model.to(device)

In [8]:
# print(model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight'])

tensor([[ 0.0023,  0.0115, -0.0081,  ...,  0.0004, -0.0018,  0.0028],
        [ 0.0031,  0.0073, -0.0066,  ...,  0.0043, -0.0025,  0.0054],
        [-0.0105, -0.0065,  0.0073,  ...,  0.0047,  0.0021, -0.0035],
        ...,
        [ 0.0356, -0.0074,  0.0166,  ...,  0.0478,  0.0349,  0.0278],
        [-0.0424,  0.0660, -0.0998,  ...,  0.0166,  0.0948, -0.0435],
        [-0.0073,  0.1032, -0.0611,  ...,  0.0290,  0.0966, -0.0325]],
       device='cuda:0')


In [6]:
optimizer = Adam(model.parameters(), lr=0.00008)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / (int(epoch/3) + 1)
)
epochs = 5


In [9]:
def model_eval(test_df) :
    model.eval()

    test_dataset = PetDataset(test_df)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

    total_loss = 0
    total_len = 0
    total_correct = 0

    for text, label in test_loader:
    #   encoded_list = [tokenizer.encode(t, add_special_token=True) for t in text]
      encoded_list = [tokenizer.encode(t, max_length=512, truncation=True) for t in text]
      padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
      sample = torch.tensor(padded_list)
      sample, label = sample.to(device), label.to(device)
      labels = torch.tensor(label)
      outputs = model(sample, labels=labels)
      _, logits = outputs

      pred = torch.argmax(F.softmax(logits), dim=1)
      correct = pred.eq(labels)
      total_correct += correct.sum().item()
      total_len += len(labels)

    print('Test accuracy: ', total_correct / total_len)  

In [12]:
# optimizer = Adam(model.parameters(), lr=0.00008)

# scheduler = lr_scheduler.LambdaLR(
#     optimizer=optimizer, lr_lambda=lambda epoch: 1 / (int(epoch/3) + 1)
# )
epochs = 10

model.train()

for epoch in range(epochs):
  total_loss = 0
  total_len = 0
  total_correct = 0
  total_count = 0
  model.train()
  for text, label in train_loader:

    encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
    padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    loss, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)
    total_loss += loss.item()
    total_count += 1
    
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()
  scheduler.step()
  model_eval(test_df)

  print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))

/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/10] Train Loss: 3.5540, Accuracy: 0.071
[Epoch 2/10] Train Loss: 3.4695, Accuracy: 0.078
[Epoch 3/10] Train Loss: 3.1790, Accuracy: 0.140
[Epoch 4/10] Train Loss: 2.6744, Accuracy: 0.267
[Epoch 5/10] Train Loss: 2.3275, Accuracy: 0.361
[Epoch 6/10] Train Loss: 2.0013, Accuracy: 0.451
[Epoch 7/10] Train Loss: 1.6922, Accuracy: 0.536
[Epoch 8/10] Train Loss: 1.4793, Accuracy: 0.604
[Epoch 9/10] Train Loss: 1.2969, Accuracy: 0.654
[Epoch 10/10] Train Loss: 1.1370, Accuracy: 0.692


In [23]:
# print(model.num_parameters())

74840083


In [12]:
# print(model.state_dict().keys())

In [14]:
model.save_pretrained("./finetune4")

In [13]:
print(model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight'])

tensor([[ 0.0097,  0.0079, -0.0082,  ...,  0.0020, -0.0014,  0.0043],
        [ 0.0108,  0.0037, -0.0068,  ...,  0.0056, -0.0020,  0.0066],
        [-0.0178, -0.0029,  0.0077,  ...,  0.0040,  0.0020, -0.0049],
        ...,
        [ 0.0278, -0.0071,  0.0190,  ...,  0.0481,  0.0346,  0.0260],
        [-0.0372,  0.0656, -0.0998,  ...,  0.0364,  0.1075, -0.0485],
        [-0.0081,  0.1068, -0.0600,  ...,  0.0492,  0.1054, -0.0366]],
       device='cuda:0')


In [ ]:
torch.save(model.state_dict(), 'tune1')

In [15]:
model.load_state_dict(torch.load('tune1'))
print(model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight'])

tensor([[ 0.0097,  0.0079, -0.0082,  ...,  0.0020, -0.0014,  0.0043],
        [ 0.0108,  0.0037, -0.0068,  ...,  0.0056, -0.0020,  0.0066],
        [-0.0178, -0.0029,  0.0077,  ...,  0.0040,  0.0020, -0.0049],
        ...,
        [ 0.0278, -0.0071,  0.0190,  ...,  0.0481,  0.0346,  0.0260],
        [-0.0372,  0.0656, -0.0998,  ...,  0.0364,  0.1075, -0.0485],
        [-0.0081,  0.1068, -0.0600,  ...,  0.0492,  0.1054, -0.0366]],
       device='cuda:0')


In [66]:
# model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=19)

In [16]:
torch.save(model.state_dict(), 'tune1')
tune1_dict = { k:v for k, v in torch.load('tune1').items() if 'classifier' not in k}
# tune1_dict = {k:v for k,v in torch.load('tune1').items()}
# print(type(tune1_dict))

In [57]:
# for k, v in torch.load('tune2').items() :
#     if 'classifier' in k :
#         print(k)
#         print(v.shape)

# #     if tune1_dict[k].shape != tune2_dict[k].shape :
#         print(k)
        

In [16]:
# # for k, v in model.state_dict() :
# #     print(parameter)
# model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight']

In [17]:
pretrained_path = './pretrained'
tune2_model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=10)
print(tune2_model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight'])

Some weights of the model checkpoint at ./pretrained were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./pretrained and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.

tensor([[ 0.0023,  0.0115, -0.0081,  ...,  0.0004, -0.0018,  0.0028],
        [ 0.0031,  0.0073, -0.0066,  ...,  0.0043, -0.0025,  0.0054],
        [-0.0105, -0.0065,  0.0073,  ...,  0.0047,  0.0021, -0.0035],
        ...,
        [ 0.0356, -0.0074,  0.0166,  ...,  0.0478,  0.0349,  0.0278],
        [-0.0424,  0.0660, -0.0998,  ...,  0.0166,  0.0948, -0.0435],
        [-0.0073,  0.1032, -0.0611,  ...,  0.0290,  0.0966, -0.0325]])


In [18]:

torch.save(tune2_model.state_dict(), 'tune2')
tune2_dict = {k:v for k,v in torch.load('tune2').items()}

tune2_dict.update(tune1_dict)
tune2_model.load_state_dict(tune2_dict)
print(tune2_model.state_dict()['roberta.encoder.layer.0.attention.self.query.weight'])

tensor([[ 0.0097,  0.0079, -0.0082,  ...,  0.0020, -0.0014,  0.0043],
        [ 0.0108,  0.0037, -0.0068,  ...,  0.0056, -0.0020,  0.0066],
        [-0.0178, -0.0029,  0.0077,  ...,  0.0040,  0.0020, -0.0049],
        ...,
        [ 0.0278, -0.0071,  0.0190,  ...,  0.0481,  0.0346,  0.0260],
        [-0.0372,  0.0656, -0.0998,  ...,  0.0364,  0.1075, -0.0485],
        [-0.0081,  0.1068, -0.0600,  ...,  0.0492,  0.1054, -0.0366]])


In [ ]:
device = torch.device("cuda")
tune2_model.to(device)

In [20]:
train_df = pd.read_csv('./files/train2.csv')
test_df = pd.read_csv('./files/test2.csv')
train_df.head()

,질병명,주진단코드,SE,진단코드,label_id
0,소화관 이물 / 섭취,2046,month 전에 자두씨 먹음 금 or 토 자두씨 먹은 다음날 회 구토 오늘 오후 ...,소화관 이물,2
1,소화관 이물 / 섭취,2046,money 선납하셨습니다 그림 cc 구토 입양하셨습니다. 펫샵 본인들이 차 종합백...,소화관 이물,2
2,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자),2001,강북동물병원 작년겨울부터 기침증상이 있었던 아이입니다. 단순 감기에 준해 치료를 하...,심장질환,4
3,소화관 이물 / 섭취,2046,money 선결제하심 정원 초콜렛 섭취. 손바닥만큼의 가나 초콜렛바 하나 다 먹음...,소화관 이물,2
4,치주 질환 / 치주염 (젖니 유전자 좌로 유래하는 것 포함),2170,money 선결제 정원 식욕 활력 양호 배변 배뇨 양호 d none 기침 콧물 n...,치주염,9


In [21]:
train_dataset = PetDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

In [33]:
# device = torch.device("cuda")
# # device = torch.device('cpu')
# pretrained_path = './finetune3'
# # pretrained_path = './pretrained'
# tokenizer = RobertaTokenizer.from_pretrained(pretrained_path, do_lower_case=False)
# model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=10)
# model.to(device)

In [22]:
optimizer = Adam(model.parameters(), lr=0.00008)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / (int(epoch/1.5) + 1)
)
epochs = 10


In [23]:
# optimizer = Adam(model.parameters(), lr=0.00008)

# scheduler = lr_scheduler.LambdaLR(
#     optimizer=optimizer, lr_lambda=lambda epoch: 1 / (int(epoch/3) + 1)
# )
# epochs = 10

model.train()

for epoch in range(epochs):
  total_loss = 0
  total_len = 0
  total_correct = 0
  total_count = 0
  model.train()
  for text, label in train_loader:

    encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
    padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    loss, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)
    total_loss += loss.item()
    total_count += 1
    
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()
  scheduler.step()
  model_eval(test_df)

  print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))

/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon2/vtdeep/petcharts/pet-env/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an 

Test accuracy:  0.5441176470588235
[Epoch 1/10] Train Loss: 2.0548, Accuracy: 0.345
Test accuracy:  0.6512605042016807
[Epoch 2/10] Train Loss: 1.2418, Accuracy: 0.592
Test accuracy:  0.6827731092436975
[Epoch 3/10] Train Loss: 0.8260, Accuracy: 0.723
Test accuracy:  0.6701680672268907
[Epoch 4/10] Train Loss: 0.6061, Accuracy: 0.804
Test accuracy:  0.707983193277311
[Epoch 5/10] Train Loss: 0.4793, Accuracy: 0.848
Test accuracy:  0.6743697478991597
[Epoch 6/10] Train Loss: 0.3664, Accuracy: 0.883
Test accuracy:  0.6869747899159664
[Epoch 7/10] Train Loss: 0.2939, Accuracy: 0.907
Test accuracy:  0.6764705882352942
[Epoch 8/10] Train Loss: 0.2416, Accuracy: 0.927
Test accuracy:  0.6743697478991597
[Epoch 9/10] Train Loss: 0.1915, Accuracy: 0.944
Test accuracy:  0.6848739495798319
[Epoch 10/10] Train Loss: 0.1639, Accuracy: 0.952
